In [ ]:
# Import necessary libraries and modules
import os
import glob
import pickle
import numpy
from music21 import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from google.colab import drive

# Define the MusicGenerator class
class MusicGenerator:
    def __init__(self, genre_id, weight_name="weights.hdf5"):
        # Initialize the MusicGenerator with a specific genre and weight name
        self.genre_id = genre_id
        self.weight_name = weight_name

        # Define genre folders for different genres
        self.genre_folders = {
            1: "/content/drive/MyDrive/project/classic",
            2: "/content/drive/MyDrive/project/movie",
            3: "/content/drive/MyDrive/project/pop",
            4: "/content/drive/MyDrive/project/rock"
        }
        # Set the genre folder based on the provided genre_id
        self.genre_folder = self.genre_folders.get(genre_id)
        if self.genre_folder is None:
            raise ValueError("Invalid genre ID. Choose a valid genre.")

        # Mount Google Drive to access files
        drive.mount('/content/drive')
        self.drive_path = self.genre_folder

    # Function to extract data from MIDI files
    def get_data(self):
        data = []
        for file in glob.glob(os.path.join(self.drive_path, "*.mid")):
            aud = converter.parse(file)
            print('Midifiles:', file)
            datatp = None
            try:
                instru = instrument.partitionByInstrument(aud)
                datatp = instru.parts[0].recurse()
            except:
                datatp = aud.flat.data
            for element in datatp:
                if isinstance(element, note.Note):
                    data.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    data.append('.'.join(str(n) for n in element.normalOrder))

        # Save the extracted data to a pickle file
        path = open(f'/content/drive/MyDrive/project/model/training_data', 'wb')
        pickle.dump(data, path)
        print(path, data)
        return data

    # Function to create sequences from the data
    def sequence(self, data, amount_of_pitch):
        squelength = 100
        pitch = sorted(set(item for item in data))
        daoin = dict((note, number) for number, note in enumerate(pitch))
        niput = []
        output = []
        for i in range(0, len(data) - squelength, 1):
            squein = data[i:i + squelength]
            squeout = data[i + squelength]
            niput.append([daoin[char] for char in squein])
            output.append([daoin[squeout]])
        ilength = len(niput)
        niput = numpy.reshape(niput, (ilength, squelength, 1))
        niput = niput / float(amount_of_pitch)
        output = to_categorical(output)
        return niput, output

    # Function to create the LSTM model
    def create_model(self, niput, amount_of_pitch):
        model = Sequential()
        model.add(LSTM(512, input_shape=(niput.shape[1], niput.shape[2]), recurrent_dropout=0.3, return_sequences=True))
        model.add(LSTM(512, recurrent_dropout=0.3, return_sequences=True,))
        model.add(LSTM(512))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(256))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(amount_of_pitch))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        return model

    # Function to train the LSTM model
    def train_network(self, model, niput, output):
        fp = f"/content/drive/MyDrive/project/weight/{self.weight_name}"
        checkpoint = ModelCheckpoint(
            fp,
            monitor='loss',
            verbose=0,
            save_best_only=True,
            mode='min'
        )
        callbacks_list = [checkpoint]
        model.fit(niput, output, epochs=10, batch_size=128, callbacks=callbacks_list)

    # Function to execute all steps
    def get_all_function(self):
        data = self.get_data()
        amount_of_pitch = len(set(data))
        niput, output = self.sequence(data, amount_of_pitch)
        model = self.create_model(niput, amount_of_pitch)
        self.train_network(model, niput, output)

# Main block
if __name__ == '__main__':
    # Get user input for genre ID
    genre_id = int(input("Enter the genre ID (1 for classic, 2 for movie, 3 for pop, 4 for rock): "))

    # Create an instance of MusicGenerator and execute all functions
    music_generator = MusicGenerator(genre_id)
    music_generator.get_all_function()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Midifiles: /content/drive/MyDrive/project/classic/alb_esp5.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_esp4.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_esp2.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se6.mid
Midifiles: /content/drive/MyDrive/project/classic/bach_850.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se3.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se2.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se7.mid
Midifiles: /content/drive/MyDrive/project/classic/bach_847.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_esp1.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se8.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se5.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_esp6.mid
Midifiles: /content/drive/MyDrive/project/classic/alb_se4.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


92/92 [==============================] - 1422s 15s/step - loss: 4.8158
Epoch 3/10
92/92 [==============================] - 1430s 16s/step - loss: 4.6920
Epoch 4/10
92/92 [==============================] - 1432s 16s/step - loss: 4.5979
Epoch 5/10
92/92 [==============================] - 1427s 16s/step - loss: 4.5143
Epoch 6/10
92/92 [==============================] - 1436s 16s/step - loss: 4.4502
Epoch 7/10
92/92 [==============================] - 1420s 15s/step - loss: 4.3895
Epoch 8/10
92/92 [==============================] - 1390s 15s/step - loss: 4.3350
Epoch 9/10
92/92 [==============================] - 1382s 15s/step - loss: 4.2877
Epoch 10/10
92/92 [==============================] - 1384s 15s/step - loss: 4.2353
